In [3]:
# Task A – Determine flight path bounding box
import time
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString
import geopandas as gpd
import os

# --- Paths ---
pos_path = "20200910_BlakesOpening_site3_flight1z.pos"
gpkg_path = "uas_flight.gpkg"
layer_name = "flight_path"

t0 = time.perf_counter()

# --- Read .pos file ---
col_names = [
    "timestamp", "lat", "lon", "height_m",
    "Q", "Q_text", "sats",
    "SDNorth_m", "SDEast_m", "SDHeight_m", "StdDev_m"
]

df = pd.read_csv(
    pos_path,
    comment="%",        # skip RTKLIB header rows
    header=None,
    names=col_names,
    dtype={
        "lat": float, "lon": float, "height_m": float,
        "Q": "Int64", "sats": "Int64",
        "SDNorth_m": float, "SDEast_m": float, "SDHeight_m": float, "StdDev_m": float
    }
)

read_time = time.perf_counter() - t0

# Drop rows without coordinates
df = df.dropna(subset=["lat", "lon"]).reset_index(drop=True)

# --- Create GeoDataFrame in WGS84 (EPSG:7844) and project to MGA2020 / Zone 55 (EPSG:7855) ---
gdf_wgs84 = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df["lon"], df["lat"])],
    crs="EPSG:7844"
)
gdf = gdf_wgs84.to_crs("EPSG:7855")

# --- Bounding box + height range ---
minx, miny, maxx, maxy = gdf.total_bounds
min_h = float(np.nanmin(gdf["height_m"]))
max_h = float(np.nanmax(gdf["height_m"]))

# --- LineString and length ---
line = LineString(list(gdf.geometry.values))
line_gdf = gpd.GeoDataFrame(
    {"total_length_m": [line.length]},
    geometry=[line],
    crs=gdf.crs
)

# Save to GeoPackage
if os.path.exists(gpkg_path):
    os.remove(gpkg_path)
line_gdf.to_file(gpkg_path, layer=layer_name, driver="GPKG")

calc_time = time.perf_counter() - t0 - read_time

# --- Nicely formatted printouts ---
def fmt_m(x): return f"{x:,.3f} m"

print("A) Determine flight path bounding box — outputs")
print("------------------------------------------------")
print("Bounding Box (EPSG:7855):")
print(f"  Easting min:  {fmt_m(minx)}")
print(f"  Easting max:  {fmt_m(maxx)}")
print(f"  Northing min: {fmt_m(miny)}")
print(f"  Northing max: {fmt_m(maxy)}")
print("Heights:")
print(f"  Height min:   {fmt_m(min_h)}")
print(f"  Height max:   {fmt_m(max_h)}")
print("Timings:")
print(f"  Read .pos:          {read_time:.3f} s")
print(f"  Calculations total: {calc_time:.3f} s")
print(f"Total flight path length: {fmt_m(line.length)}")
print(f"GeoPackage written: {gpkg_path} (layer: {layer_name})")


C:\Users\User\anaconda3\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


A) Determine flight path bounding box — outputs
------------------------------------------------
Bounding Box (EPSG:7855):
  Easting min:  467,724.652 m
  Easting max:  468,410.536 m
  Northing min: 5,225,706.246 m
  Northing max: 5,226,080.720 m
Heights:
  Height min:   281.855 m
  Height max:   304.533 m
Timings:
  Read .pos:          1.912 s
  Calculations total: 2.451 s
Total flight path length: 3,663.539 m
GeoPackage written: uas_flight.gpkg (layer: flight_path)


In [5]:
df["SDPlanimetric_m"] = np.sqrt(df["SDEast_m"]**2 + df["SDNorth_m"]**2)

print(df["SDPlanimetric_m"])

import matplotlib as plt
import pathlib as path

plt.subplot

0        0.029698
1        0.009220
2        0.009220
3        0.009220
4        0.009220
           ...   
14360    0.007810
14361    0.007810
14362    0.008602
14363    0.010000
14364    0.010000
Name: SDPlanimetric_m, Length: 14365, dtype: float64
